# Session 2
## Model options

In [2]:
import pybamm

The simplest (and often best) way to change submodels is to pass options to the model. Let's load up the default DFN and the DFN with a lumped thermal model. We can do this by passing a dictionary of options to the model when we create it. We can take a look at the documentation to see the available model options.

In [3]:
dfn = pybamm.lithium_ion.DFN(name="DFN")
tdfn = pybamm.lithium_ion.DFN(options={"thermal":"lumped"}, name="TDFN")

If we take a look at the submodels for the negative particle, for example, we see they are the same

In [4]:
print(dfn.submodels["negative particle"])
print(tdfn.submodels["negative particle"])

but the thermal submodel is different 

In [5]:
print(dfn.submodels["thermal"])
print(tdfn.submodels["thermal"])

Let's solve the models and compare the results

In [6]:
models = [dfn, tdfn]

params = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Chen2020)

sols = []
for model in models:
    sim = pybamm.Simulation(model, parameter_values=params)
    sol = sim.solve([0, 3600])
    sols.append(sol)

In [7]:
pybamm.dynamic_plot(sols, ["Terminal voltage [V]", "Volume-averaged cell temperature [K]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=3560.9883224964774, step=35.609883224964776)…

As well as using submodels to add new physics (e.g. thermal effects or degradation models), we can use submodels to change the way in which certain effects are modelled. For example, the default option for the particle concentrations numerically solves a PDE describing nonlinear Fickian diffusion, but instead of solving the PDE numerically we could approximate the solution by assuming a polynomial concentration profile. Both options assume Fickian diffusion, but solve the problem in different ways. 

Let's compare the two approaches

In [8]:
models = [
    pybamm.lithium_ion.DFN(name="PDE"),
    pybamm.lithium_ion.DFN(options={"particle":"quadratic profile"}, name="quadratic")
]

sols = []
for model in models:
    sim = pybamm.Simulation(model)
    sol = sim.solve([0, 3600])
    sols.append(sol)

In [9]:
pybamm.dynamic_plot(sols, ["X-averaged negative particle concentration", "Terminal voltage [V]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

This does a pretty good job. Let's try using a parameter set where the diffusion is nonlinear

In [ ]:
models = [
    pybamm.lithium_ion.DFN(name="PDE"),
    pybamm.lithium_ion.DFN(options={"particle":"quadratic profile"}, name="quadratic")
]

# use Ecker parameters where diffusion is highly nonlinear
params = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Ecker2015)

sols = []
for model in models:
    sim = pybamm.Simulation(model, parameter_values=params)
    sol = sim.solve([0, 3600])
    sols.append(sol)

In [ ]:
pybamm.dynamic_plot(sols, ["X-averaged negative particle concentration", "Terminal voltage [V]"])

We still do a pretty good job at getting the correct voltage, but do less well at matching the concentration profile within the particle.